In [ ]:
import h5py
import numpy as np
import pandas as pd
import tables
import os
import sys
import time
import glob
import scipy.io as sio
import hdf5_getters
from pandas import read_hdf
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
finalWorkingDF = pd.read_pickle("finalWorkingDF")

In [ ]:
# Separate data into training and testing set
y = finalWorkingDF['year']
x = finalWorkingDF.drop('year', axis = 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
x_train.to_pickle("XTrain")
y_train.to_pickle("YTrain")
x_test.to_pickle("XTest")
y_test.to_pickle("YTest")

In [ ]:
x_trainNumPy = x_train.to_numpy()
x_testNumPy = x_test.to_numpy()
y_trainNumPy = y_train.to_numpy()
y_testNumPy = y_test.to_numpy()

In [ ]:
###################### KNN CLASSIFIER ########################
from sklearn.metrics import mean_squared_error
n_neighbors = 15

nnClassifier = neighbors.KNeighborsClassifier(n_neighbors)
nnClassifier.fit(x_trainNumPy, y_trainNumPy)

# Get training and test predictions
predictedtrainyears = nnClassifier.predict(x_trainNumPy)
trainMSE = mean_squared_error(y_trainNumPy, predictedtrainyears)
testMSE = mean_squared_error(y_testNumPy, nnClassifier.predict(x_testNumPy))
# classifier_trainMSE = np.mean((nnClassifier.predict(x_trainNumPy) - y_trainNumPy)**2)
# classifier_testMSE = np.mean((nnClassifier.predict(x_testNumPy) - y_testNumPy)**2)

print("training MSE", trainMSE)
print("testing MSE", testMSE)

In [ ]:
###################### KNN REGRESSION ########################
n_neighbors = 15

nnRegressor = neighbors.KNeighborsRegressor(n_neighbors)
nnRegressor.fit(x_trainNumPy, y_trainNumPy)

# Get training and test predictions
predictedtrainyears = nnRegressor.predict(x_trainNumPy)
trainMSE = mean_squared_error(y_trainNumPy, predictedtrainyears)
testMSE = mean_squared_error(y_testNumPy, nnRegressor.predict(x_testNumPy))

print("training MSE", trainMSE)
print("testing MSE", testMSE)

In [ ]:
###################### NEURAL NETWORK ########################
# create model
model = Sequential()
model.add(Dense(12, input_shape=(3744,37), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the model
model.fit(x_trainNumPy, y_trainNumPy, epochs=150, batch_size=10)

# evaluate the model
train_scores = model.evaluate(x_trainNumPy, y_trainNumPy)
print("\n%s: %.2f%%" % (model.metrics_names[1], train_scores[1]*100))
test_scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

**10-701 Song Project**

**Introduction**
For this project, we are exploring the question of whether different data sets about songs can predict the year of the songs' origin equally well. In particular we are looking at the 10,000 song subset of the million song dataset and the musixmatch dataset for the 10,000 song subset. The 10,000 song subset contains very detailed information about each song such as timbre and pitch vectors for each segment of the song, tatums, loudness and various other algorithmically determined factors such as artist hotness, song hotness, danceability, etc. In contrast, the musixmatch dataset only contains bag-of-words lyrics for each song. Our goal is to determine whether models predicting a song's release year trained on the musixmatch dataset can exhibit similar testing performance as models trained on the full feature-set of the 10k subset.